In [2]:
import pandas as pd
og = pd.read_csv("../data/adjusted/augmented_full.csv")

#num updates
updates = pd.read_json("../data/updates.json")
updates = updates[updates.id==updates.parent].pitch.str.split("/updates",n=0).map(lambda x:x[0]).value_counts().reset_index().rename({"pitch":"url","count":"post_count"},axis=1)
og = og.merge(updates,on="url",how="left")
og["post_count"].fillna(0,inplace=True)

In [3]:
comp = og.sort_values("Date")[["id","Date","company"]]
comp = pd.concat((comp,comp.groupby("company").cumcount().map(bool).rename("serial")),axis=1)
og = og.merge(comp,on=["id","Date"])

In [4]:
og[og.columns[og.columns.str.contains("company")]]

,company_x,company.company_industry,company.company_industry.id,company.company_industry.name,company.company_industry.slug,company.company_number,company.created_at,company.facebook,company.growth_stage,company.id,...,company.marketplace_slug,company.name,company.slug,company.status,company.status_notes,company.summary,company.twitter,company.updated_at,company.website,company_y
0,https://www.crowdcube.eu/companies/cutbox-barbers,NaN,2.0,"Apparel & accessories, home and personal",apparel-accessories-home-personal,12541816,2023-04-24 09:26:18,https://www.facebook.com/CutboxBarbers,NaN,bkMJjl,...,NaN,CUTBOX LIMITED,cutbox-barbers,trading,NaN,NaN,NaN,2023-06-12 14:35:56,http://cutbox.co.uk,https://www.crowdcube.eu/companies/cutbox-barbers
1,https://www.crowdcube.eu/companies/hooked-foods,NaN,12.0,Food & beverage (FMCG),food-beverage,559161-9944,2022-06-09 09:50:33,https://www.facebook.com/hookedfoods,NaN,qnxABZ,...,NaN,Hooked Foods AB,hooked-foods,trading,NaN,NaN,NaN,2023-07-13 10:53:10,https://hookedfoods.com/,https://www.crowdcube.eu/companies/hooked-foods
2,https://www.crowdcube.eu/companies/mapal-aerat...,NaN,8.0,Energy and renewables,energy-renewables,12037842,2022-11-15 13:54:02,https://www.facebook.com/mapal-green-energy-ltd,NaN,bw3Kob,...,NaN,MAPAL AERATION LIMITED,mapal-aeration-limited,trading,NaN,NaN,https://twitter.com/MapalGreen,2023-04-17 13:27:28,https://www.mapal-ge.com,https://www.crowdcube.eu/companies/mapal-aerat...
3,https://www.crowdcube.eu/companies/move-techno...,NaN,NaN,NaN,NaN,13904894,2023-06-08 09:16:33,NaN,NaN,lxA53Z,...,NaN,Move Technologies Group Ltd,move-technologies-group-ltd,trading,NaN,NaN,NaN,2023-06-27 15:59:41,http://www.testmovetech.com,https://www.crowdcube.eu/companies/move-techno...
4,https://www.crowdcube.eu/companies/move-techno...,NaN,NaN,NaN,NaN,13904894,2023-06-08 09:16:33,NaN,NaN,lxA53Z,...,NaN,Move Technologies Group Ltd,move-technologies-group-ltd,trading,NaN,NaN,NaN,2023-06-27 15:59:41,http://www.testmovetech.com,https://www.crowdcube.eu/companies/move-techno...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1590,https://www.crowdcube.eu/companies/kamm-sons,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.crowdcube.eu/companies/kamm-sons
1591,https://www.crowdcube.eu/companies/kamm-sons,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.crowdcube.eu/companies/kamm-sons
1592,https://www.crowdcube.eu/companies/the-rushmor...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.crowdcube.eu/companies/the-rushmor...
1593,https://www.crowdcube.eu/companies/the-rushmor...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.crowdcube.eu/companies/the-rushmor...


In [5]:
launch = og["dates.activated"]
last_change = (og[og.columns[og.columns.str.contains("update")]].apply(pd.to_datetime).min(axis=1,skipna=True))
changed = launch < last_change
changed[launch.isna() & last_change.isna()] = pd.NA
og["updated"] = changed

/var/folders/t0/lg3jmzv573q7n1cv7kt891y40000gn/T/ipykernel_46503/1911553717.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'nan' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  changed[launch.isna() & last_change.isna()] = pd.NA


In [6]:
og

,Unnamed: 0,id,name,url,Date,dates.activated,dates.funded,dates.closed,investment.current,investment.current_adj_EUR,...,venture_capital_backer,venture_capital_backing_amount,received_any_awards,award_name,received_media_coverage_and_endorsement,media_coverage_and_endorsement,post_count,company_y,serial,updated
0,0,lKnKBq,CUTBOX,https://www.crowdcube.eu/companies/cutbox-barb...,2023-12-06,2023-06-09 13:41:14,2023-06-12 08:29:49,2023-07-05 00:15:06,491242.90,573359.44,...,Fuel Ventures,unknown,False,NaN,False,NaN,15.0,https://www.crowdcube.eu/companies/cutbox-barbers,False,True
1,1,bJ1kdZ,Hooked Foods,https://www.crowdcube.eu/companies/hooked-food...,2023-06-13,2023-06-13 09:16:28,2023-06-13 09:22:30,2023-07-01 00:15:11,638736.61,638736.61,...,"Oysterbay, Big Idea Ventures & Brightly",5M€,False,NaN,False,NaN,9.0,https://www.crowdcube.eu/companies/hooked-foods,False,True
2,2,qaE20q,Mapal Aeration Solutions,https://www.crowdcube.eu/companies/mapal-aerat...,2023-06-13,2023-05-03 13:48:47,2023-06-13 18:50:45,2023-06-21 00:15:08,463814.41,540261.40,...,NaN,NaN,False,NaN,False,NaN,22.0,https://www.crowdcube.eu/companies/mapal-aerat...,False,False
3,3,bvLKxl,Move Technologies Group Ltd,https://www.crowdcube.eu/companies/move-techno...,2023-09-06,NaN,2023-06-09 08:17:41,2023-06-09 08:17:41,57200.00,66670.55,...,ABC Ventures,$10 million,False,NaN,True,Featured in TechCrunch,0.0,https://www.crowdcube.eu/companies/move-techno...,True,False
4,4,q4Mwol,Move Technologies Group Ltd,https://www.crowdcube.eu/companies/move-techno...,2023-09-06,NaN,2023-06-09 07:48:28,2023-06-09 07:48:28,932850.00,1087301.12,...,ABC Ventures,$10 million,False,NaN,True,Featured in TechCrunch,0.0,https://www.crowdcube.eu/companies/move-techno...,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1590,1590,Kammerling's Investment Holdings Ltd_0,Kammerling's Investment Holdings Ltd,https://www.crowdcube.eu/investment/kamm-sons-...,2012-12-20,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,False,NaN,False,NaN,0.0,https://www.crowdcube.eu/companies/kamm-sons,True,NaN
1591,1591,Kammerling's_0,Kammerling's,https://www.crowdcube.eu/investment/kamm-sons-...,2011-10-13,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,False,NaN,False,NaN,0.0,https://www.crowdcube.eu/companies/kamm-sons,False,NaN
1592,1592,The Rushmore Group Ltd_0,The Rushmore Group Ltd,https://www.crowdcube.eu/investment/the-rushmo...,2013-04-15,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,False,NaN,False,NaN,0.0,https://www.crowdcube.eu/companies/the-rushmor...,True,NaN
1593,1593,The Rushmore Group Ltd - Extended offer_0,The Rushmore Group Ltd - Extended offer,https://www.crowdcube.eu/investment/the-rushmo...,2011-12-21,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,False,NaN,False,NaN,0.0,https://www.crowdcube.eu/companies/the-rushmor...,True,NaN


In [7]:
og[og.name.str.contains("cutbox",case=False)].to_dict()

{'Unnamed: 0': {0: 0},
 'id': {0: 'lKnKBq'},
 'name': {0: 'CUTBOX'},
 'url': {0: 'https://www.crowdcube.eu/companies/cutbox-barbers/pitches/lKnKBq'},
 'Date': {0: '2023-12-06'},
 'dates.activated': {0: '2023-06-09 13:41:14'},
 'dates.funded': {0: '2023-06-12 08:29:49'},
 'dates.closed': {0: '2023-07-05 00:15:06'},
 'investment.current': {0: 491242.9},
 'investment.current_adj_EUR': {0: 573359.44},
 'investment.maximum': {0: 400000.0},
 'investment.maximum_adj_EUR': {0: 466864.31},
 'investment.minimum': {0: 33.26},
 'investment.minimum_adj_EUR': {0: 38.82},
 'investment.multiple': {0: 33.25721},
 'investment.multiple_adj_EUR': {0: 38.82},
 'investment.target': {0: 400000.0},
 'investment.target_adj_EUR': {0: 466864.31},
 'investment_fee.maximum': {0: 250.0},
 'investment_fee.maximum_adj_EUR': {0: 291.79},
 'investment_fee.minimum': {0: 2.49},
 'investment_fee.minimum_adj_EUR': {0: 2.91},
 'stats.investments.daily_average': {0: 8931.0},
 'stats.investments.daily_average_adj_EUR': {0: 10

In [8]:
og[og.name.str.contains("cutbox",case=False)].to_dict()

{'Unnamed: 0': {0: 0},
 'id': {0: 'lKnKBq'},
 'name': {0: 'CUTBOX'},
 'url': {0: 'https://www.crowdcube.eu/companies/cutbox-barbers/pitches/lKnKBq'},
 'Date': {0: '2023-12-06'},
 'dates.activated': {0: '2023-06-09 13:41:14'},
 'dates.funded': {0: '2023-06-12 08:29:49'},
 'dates.closed': {0: '2023-07-05 00:15:06'},
 'investment.current': {0: 491242.9},
 'investment.current_adj_EUR': {0: 573359.44},
 'investment.maximum': {0: 400000.0},
 'investment.maximum_adj_EUR': {0: 466864.31},
 'investment.minimum': {0: 33.26},
 'investment.minimum_adj_EUR': {0: 38.82},
 'investment.multiple': {0: 33.25721},
 'investment.multiple_adj_EUR': {0: 38.82},
 'investment.target': {0: 400000.0},
 'investment.target_adj_EUR': {0: 466864.31},
 'investment_fee.maximum': {0: 250.0},
 'investment_fee.maximum_adj_EUR': {0: 291.79},
 'investment_fee.minimum': {0: 2.49},
 'investment_fee.minimum_adj_EUR': {0: 2.91},
 'stats.investments.daily_average': {0: 8931.0},
 'stats.investments.daily_average_adj_EUR': {0: 10

In [9]:
basic = ['id', 'name', 'url', 'Date', 'dates.activated','dates.funded', 'dates.closed',]

In [10]:
og['company.company_industry.name'].unique().tolist()

['Apparel & accessories, home and personal',
 'Food & beverage (FMCG)',
 'Energy and renewables',
 nan,
 'Healthtech & healthcare',
 'Advertising, marketing & promotion',
 'Financial services & payments',
 'Automotive, transport and mobility',
 'Leisure, hospitality & tourism',
 'Building, property & land management',
 'Entertainment & media',
 'Restaurants, cafes and bars',
 'Business Services, IT & Telecomms',
 'Education and training',
 'Manufacturing/R&D',
 'Fitness & sports',
 'Analytics, Ai, data, Bi, automation and robotics',
 'Lifesciences & biotech',
 'Recruitment & procurement',
 'Legal, compliance and security']

In [11]:
tech=[#tech
'Analytics, Ai, data, Bi, automation and robotics',
'Business Services, IT & Telecomms',
#maybe tech
'Healthtech & healthcare',
'Lifesciences & biotech',
'Energy and renewables',
]

non_tech=[
#maybe non-tech
'Financial services & payments',
'Automotive, transport and mobility',
#non-tech
'Entertainment & media',
'Apparel & accessories, home and personal',
'Food & beverage (FMCG)',
'Advertising, marketing & promotion',
'Leisure, hospitality & tourism',
'Building, property & land management',
'Restaurants, cafes and bars',
'Education and training',
'Manufacturing/R&D',
'Fitness & sports',
'Recruitment & procurement',
'Legal, compliance and security']



In [12]:
vars = ['Sustainability_Oriented',
 'success_target',
 'success_percentaige_raised (in %)',
 'Funding_Amount (%)',
 'No_Investors (in EUR)',
 'Average pledge (Euro)',
 'Speed of investment',
 'Social Capital_LinkedIn',
 'Gender',
 'education',
 'Social media',
 'Owner’s educational level',
 'Grant',
 'Award',
 'endorsements',
 'Prior experience',
 'third-party endorsement',
 'Tech industry',
 'Sustainability_Oriented',
 'Founder_Experience',
 'founders education',
 'Advisors',
 'Retained equity (in %)',
 'valuation',
 'Pure equity',
 'IPR',
 'Tax_Incentives (%)',
 'EIS tax relief',
 'SEIS tax relief',
 'Country',
 'Video ',
 'Business angel backing',
 'updates posted',
 'Target_Capital',
 'Serial',
 'Duration (days)',
 'Updated',
 'Pre-money valuation',
 'Campaign goal',
 'Max pledge',
 'Max pledge / goal',
 'government grant',
 'Equity offered',
 'company age',
 'sector',
 'team size',
 'rounds ',
 'Tax_Incentives',
 'VC_backing',
 'partnerships']

In [13]:
pd.to_datetime(og["dates.closed"])-pd.to_datetime(og["dates.activated"])

0      25 days 10:33:52
1      17 days 14:58:43
2      48 days 10:26:21
3                   NaT
4                   NaT
             ...       
1590                NaT
1591                NaT
1592                NaT
1593                NaT
1594                NaT
Length: 1595, dtype: timedelta64[ns]

In [14]:
import pandas as pd
from w3lib.html import remove_tags

teams = pd.read_pickle("./extracted.pkl").reset_index(drop=True)
teams = teams.drop_duplicates(subset="input")
#teams["team_names"] = person.map(lambda x:x["Person Name"],na_action="ignore").groupby(level=0).apply(set).apply(list)
#teams["team_size"] = teams.team_names.map(len)
#teams


teams = teams.explode("text")
teams.input = teams.input.map(remove_tags, na_action="ignore")
teams = pd.concat([teams.reset_index(drop=True),pd.json_normalize(teams.text).reset_index(drop=True)],axis=1)
teams = teams.merge(og[["id","name","url","Date","the_people.content"]],right_on="the_people.content",left_on="input",how="left")
teams = teams.rename({"id":"project_id","name":"project_name"},axis=1)
teams = teams[['project_id', 'project_name', 'Date','Person Name', 'Job Title']]
teams

,project_id,project_name,Date,Person Name,Job Title
0,qaE20q,Mapal Aeration Solutions,2023-06-13,Zeev Fisher,CEO and co-founder
1,qaE20q,Mapal Aeration Solutions,2023-06-13,Julian Lowe,UK Sales Manager
2,qaE20q,Mapal Aeration Solutions,2023-06-13,Ilan Diamond,Executive Chairman
3,qaE20q,Mapal Aeration Solutions,2023-06-13,Elad Nahmias,VP of Operations
4,qaE20q,Mapal Aeration Solutions,2023-06-13,Amir Zemer,Engineering and Process Manager
...,...,...,...,...,...
6485,l8nrkb,CROWDCUBE,2016-07-18,Paul Massey,General Counsel
6486,l8nrkb,CROWDCUBE,2016-07-18,Tim Bunting,Non-Executive Director
6487,l8nrkb,CROWDCUBE,2016-07-18,Tim Draper,Advisor
6488,l8nrkb,CROWDCUBE,2016-07-18,Simon Cook,Advisor


In [15]:
teams_new = pd.read_csv("../data/adjusted/team_members.csv")
teams_new["Person Name"] = teams_new[["first_name","last_name"]].agg(" ".join,axis=1)
teams_new = teams_new.rename({"job_title":"Job Title"},axis=1)
teams_new = teams_new.merge(og[["name","id","Date"]],left_on="project_id",right_on="id",suffixes=("","_"))
teams_new = teams_new[['name', 'project_id', 'Date', 'Person Name', 'Job Title', 'id', 'linkedin_url']].rename({'name':"project_name", 'id':"person_id",},axis=1)
teams_new

,project_name,project_id,Date,Person Name,Job Title,person_id,linkedin_url
0,CUTBOX,lKnKBq,2023-12-06,Oz Oboh,Co-Founder & CEO,EbMN5q,https://www.linkedin.com/in/osajie-oboh-b5291a...
1,CUTBOX,lKnKBq,2023-12-06,Louis Angel Scott,Co-Founder & COO,9Z7yNq,https://www.linkedin.com/in/louis-angel-scott/
2,CUTBOX,lKnKBq,2023-12-06,James Crowson,Head of Finance,PbvgGq,https://www.linkedin.com/in/james-crowson-8409...
3,CUTBOX,lKnKBq,2023-12-06,Dorian Waite,Advisor,ebw88q,https://www.linkedin.com/in/dorian-waite-9207b...
4,CUTBOX,lKnKBq,2023-12-06,Philip Raub,Advisor,XlEVaZ,https://www.linkedin.com/in/praub/
...,...,...,...,...,...,...,...
95,Freetrade,bwL9Jq,2023-06-22,Adam Dodds,Founder and CEO,KbVxVl,https://www.linkedin.com/in/adamdodds
96,Freetrade,bwL9Jq,2023-06-22,Viktor Nebehaj,Founder and COO,4qn9Bb,https://www.linkedin.com/in/viktornebehaj
97,Freetrade,bwL9Jq,2023-06-22,Shahid Naveed,CTO,elejzZ,https://www.linkedin.com/in/shahidnaveed
98,Freetrade,bwL9Jq,2023-06-22,Paul Brooking,CFO,2bdoAZ,https://www.linkedin.com/in/paul-brooking-2a61...


In [18]:
#pd.concat([teams,teams_new]).reset_index().drop_duplicates().to_csv("team_member_augmented.csv")

In [16]:
all = pd.concat([teams,teams_new]).reset_index().drop_duplicates()
all = all.groupby(['project_id', 'project_name', 'Date'])["Person Name"].count().reset_index().rename({"Person Name":"team_size"},axis=1).sort_values("Date").reset_index(drop=True)
all

,project_id,project_name,Date,team_size
0,bj0eGl,HealthUnlocked,2016-01-11,3
1,bwaVWl,Smart Property,2016-02-09,2
2,bwaD7l,TYNE BANK BREWERY,2016-03-05,3
3,Z54o2l,Molecular Warehouse,2016-03-10,5
4,qY4yrb,Innis & Gunn,2016-03-11,3
...,...,...,...,...
1215,bdpADb,Nanusens,2023-11-05,6
1216,bgWnBb,FAITH IN NATURE,2023-11-07,7
1217,lx9kvZ,Gravitricity,2023-12-04,7
1218,lKnKBq,CUTBOX,2023-12-06,6


In [111]:
og_with_teams = og.merge(all,left_on=['id', 'name'],right_on=['project_id', 'project_name'],how="left")
og_with_teams.team_size.isna().sum()


375

In [117]:
import pandas as pd
pd.options.mode.chained_assignment = None

cc_reg = og[basic]
#cc_reg['Sustainability_Oriented'] = ? #kann weg
cc_reg['success_target'] = og["investment.percentage"] >= 100
cc_reg['success_percentaige_raised (in %)'] = og["investment.percentage"]
cc_reg['Funding_Amount (%)'] = og["investment.current_adj_EUR"].fillna(og["Amount_adj_EUR"])
cc_reg['No_Investors (in EUR)'] = og['No. of Investors']
cc_reg['Average pledge (Euro)'] = (cc_reg['Funding_Amount (%)'] / cc_reg['No_Investors (in EUR)']).round(2)
cc_reg['Speed of investment'] = pd.to_datetime(og["dates.funded"])-pd.to_datetime(og["dates.activated"])
#cc_reg['Social Capital_LinkedIn'] = 
#cc_reg['Gender'] = 
#cc_reg['education'] = 
#cc_reg['Social media'] = 
#cc_reg['Owner’s educational level'] = 
#cc_reg['Grant'] = 
cc_reg['Award'] = og["received_any_awards"]
cc_reg['endorsements'] = og["received_media_coverage_and_endorsement"]
#cc_reg['Prior experience'] = 
#cc_reg['third-party endorsement'] = 
cc_reg['Tech industry'] = og["company.company_industry.name"].isin(tech)
#cc_reg['Sustainability_Oriented'] = ?
#cc_reg['Founder_Experience'] = ?
#cc_reg['founders education'] = ?
#cc_reg['Advisors'] = ?
cc_reg['Retained equity (in %)'] =  100 - og["equity"].clip(0,100)
cc_reg['valuation'] = og["valuation.post_adj_EUR"]
cc_reg['Pure equity'] = og["equity"]
#cc_reg['IPR'] = ?
cc_reg['Tax_Incentives (%)'] = og["display_eis"]
cc_reg['EIS tax relief'] = og["eis"]
cc_reg['SEIS tax relief'] = og["seis"]
cc_reg['Country'] = og["country"]
cc_reg['Video '] = og[og.columns[og.columns.str.contains("media")]].map(str).agg(" ".join,axis=1).str.contains("'type': 'video'")
#cc_reg['Business angel backing'] = ?
cc_reg['updates posted'] = og["post_count"]
cc_reg['Target_Capital'] = og["investment.target_adj_EUR"]
cc_reg['Serial'] = og["serial"]
cc_reg['Duration (days)'] = pd.to_datetime(og["dates.closed"])-pd.to_datetime(og["dates.activated"])
cc_reg['Updated'] = og["updated"]
cc_reg['Pre-money valuation'] = og["valuation.pre_adj_EUR"]
cc_reg['Campaign goal'] = og["investment.target_adj_EUR"]
cc_reg['Max pledge'] = og["stats.investments.largest_adj_EUR"]
cc_reg['Max pledge / goal'] = cc_reg['Max pledge']/cc_reg['Campaign goal']
#cc_reg['government grant'] = ?
cc_reg['Equity offered'] = og["equity"]
cc_reg['company age'] = pd.to_datetime(og["dates.activated"])-pd.to_datetime(og["company.created_at"])
cc_reg['sector'] = og["company.company_industry.name"]
cc_reg['teamsize'] = og_with_teams["team_size"]
#cc_reg['rounds '] = 
cc_reg['Tax_Incentives'] = og["seis"]
cc_reg['VC_backing'] = og["received_venture_capital_backing"]
#cc_reg['partnerships'] = ?
cc_reg

,id,name,url,Date,dates.activated,dates.funded,dates.closed,success_target,success_percentaige_raised (in %),Funding_Amount (%),...,Pre-money valuation,Campaign goal,Max pledge,Max pledge / goal,Equity offered,company age,sector,teamsize,Tax_Incentives,VC_backing
0,lKnKBq,CUTBOX,https://www.crowdcube.eu/companies/cutbox-barb...,2023-12-06,2023-06-09 13:41:14,2023-06-12 08:29:49,2023-07-05 00:15:06,True,122.0,573359.44,...,9206422.89,466864.31,452715.98,0.969695,5.86,46 days 04:14:56,"Apparel & accessories, home and personal",6.0,no,True
1,bJ1kdZ,Hooked Foods,https://www.crowdcube.eu/companies/hooked-food...,2023-06-13,2023-06-13 09:16:28,2023-06-13 09:22:30,2023-07-01 00:15:11,True,114.0,638736.61,...,11717049.00,560000.00,258791.00,0.462127,5.17,368 days 23:25:55,Food & beverage (FMCG),6.0,no,True
2,qaE20q,Mapal Aeration Solutions,https://www.crowdcube.eu/companies/mapal-aerat...,2023-06-13,2023-05-03 13:48:47,2023-06-13 18:50:45,2023-06-21 00:15:08,True,115.0,540261.40,...,5788976.12,465928.95,354845.66,0.761587,8.54,168 days 23:54:45,Energy and renewables,5.0,no,False
3,bvLKxl,Move Technologies Group Ltd,https://www.crowdcube.eu/companies/move-techno...,2023-09-06,NaN,2023-06-09 08:17:41,2023-06-09 08:17:41,True,57200.0,66670.55,...,0.00,116.56,8741.77,74.998027,100.18,NaT,NaN,NaN,no_yes_pending,True
4,q4Mwol,Move Technologies Group Ltd,https://www.crowdcube.eu/companies/move-techno...,2023-09-06,NaN,2023-06-09 07:48:28,2023-06-09 07:48:28,True,65535.0,1087301.12,...,0.00,116.56,233113.82,1999.946980,100.01,NaT,NaN,NaN,no_yes_pending,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1590,Kammerling's Investment Holdings Ltd_0,Kammerling's Investment Holdings Ltd,https://www.crowdcube.eu/investment/kamm-sons-...,2012-12-20,NaN,NaN,NaN,False,NaN,398968.82,...,NaN,NaN,NaN,NaN,20.00,NaT,NaN,NaN,NaN,False
1591,Kammerling's_0,Kammerling's,https://www.crowdcube.eu/investment/kamm-sons-...,2011-10-13,NaN,NaN,NaN,False,NaN,205514.64,...,NaN,NaN,NaN,NaN,23.00,NaT,NaN,NaN,NaN,False
1592,The Rushmore Group Ltd_0,The Rushmore Group Ltd,https://www.crowdcube.eu/investment/the-rushmo...,2013-04-15,NaN,NaN,NaN,False,NaN,586235.20,...,NaN,NaN,NaN,NaN,4.61,NaT,NaN,NaN,NaN,False
1593,The Rushmore Group Ltd - Extended offer_0,The Rushmore Group Ltd - Extended offer,https://www.crowdcube.eu/investment/the-rushmo...,2011-12-21,NaN,NaN,NaN,False,NaN,600744.92,...,NaN,NaN,NaN,NaN,5.00,NaT,NaN,NaN,NaN,False


In [119]:
cc_reg.to_csv("../data/adjusted/final_regression.csv")